In [61]:
import os
import re
import shutil
import zipfile
import requests
import pandas as pd
import tensorflow as tf
import urllib3
import unicodedata
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, GRU, Masking
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from collections import Counter
from nltk import ngrams

## 문자(Char) level Machine Translator
seq2seq는 기계 번역에 많이 사용한다

In [3]:
# data 다운 받기
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def download_zip(url, output_path):
    response = requests.get(url, headers=headers, stream=True)
    if response.status_code == 200:
        with open(output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"ZIP file downloaded to {output_path}")
    else:
        print(f"Failed to download. HTTP Response Code: {response.status_code}")

url = "http://www.manythings.org/anki/fra-eng.zip"
output_path = "fra-eng.zip"
download_zip(url, output_path)

path = os.getcwd()
zipfilename = os.path.join(path, output_path)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)


ZIP file downloaded to fra-eng.zip


In [4]:
# 데이터를 읽어와 dataframe 생성하기
# 예제와 데이터 개수가 다름
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
print('전체 샘플의 개수 :',len(lines))

전체 샘플의 개수 : 227815


In [6]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000] # 6만개만 저장
lines.sample(10) # 10개 random 출력

,src,tar
20036,He winked at her.,Il lui a fait un clin d'œil.
30498,We're all retired.,Nous sommes toutes à la pension.
17416,They feared you.,Elles te craignaient.
15434,I have to think.,Il me faut réfléchir.
39501,Everybody hates you.,Tout le monde vous déteste.
39740,He did the opposite.,Il fit le contraire.
2696,We made it.,Nous avons réussi.
21637,I've never tried.,Je n'ai jamais essayé.
39399,Don't fret about it.,Ne te tracasse pas pour ça.
49307,I won't ever give up.,Je n'abandonnerai jamais.


In [7]:
# 시작 <sos> token과 종료 <eos> token 삽입 필요
# \t = start / \n = end 로 간주

# 모든 data 앞 뒤에 시퀀스 삽입
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

,src,tar
48823,I prefer it this way.,\t Je le préfère de cette manière. \n
41645,I'm absolutely fine.,\t Je vais parfaitement bien. \n
8738,It's very big.,\t C'est très grand. \n
58913,My cat will love this.,\t Mon chat adorera ça. \n
57311,I think you're a liar.,\t Je pense que tu es un menteur. \n
52341,Tom is philosophical.,\t Tom est philosophe. \n
37917,Where's your drink?,\t Où est ton verre ? \n
52602,Tom was hit by a car.,\t Tom a été renversé par une voiture. \n
10869,Have you eaten?,\t Avez-vous mangé ? \n
48075,I didn't expect help.,\t Je n'attendais pas d'aide. \n


In [8]:
# 문자 집합 구축
# src에서 수행
src_vocab = set() # 빈 집합 생성
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 문자씩 읽음
        src_vocab.add(char) # 집합에 문자 추가하기

# tar에서 수행
tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [9]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print('source 문장의 char 집합(영어) :',src_vocab_size)
print('target 문장의 char 집합(프랑스어) :',tar_vocab_size)

source 문장의 char 집합 : 80
target 문장의 char 집합 : 104


In [10]:
# 정렬하여 순서를 정해주고 index 사용하여 출력하기
# 막 호출하려고 하면 오류 출력
# 1차적으로 정렬하기

# 영어
src_vocab = sorted(list(src_vocab)) # 정렬!

# 프랑스어
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['T', 'U', 'V', 'W', 'X', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x']


In [12]:
# dict 생성하고 index 부여하기
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, 'ï': 77, '’': 78, '€': 79}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 3

In [13]:
# 영어 정수 인코딩 수행하기
encoder_input = []

# 목적: 영어(src) columns를 돌아가며 1개의 문장 씩 정수 인코딩을 수행하여 리스트에 넣는다.
for line in lines.src:
  encoded_line = []
  # 각 줄에서 1개의 char 본다
  for char in line:
    # 각 char을 정수로 변환하여 리스트를 넣는다
    encoded_line.append(src_to_index[char])
  encoder_input.append(encoded_line)
print('source 문장의 정수 인코딩 :',encoder_input[:5])

source 문장의 정수 인코딩 : [[30, 64, 10], [30, 64, 10], [30, 64, 10], [30, 64, 10], [31, 58, 10]]


In [14]:
# 프랑스어 정수 인코딩 수행하기 - 영어와 동일, 데이터만 다름
decoder_input = []

for line in lines.tar:
  encoded_line = []
  for char in line:
    encoded_line.append(tar_to_index[char])
  decoder_input.append(encoded_line)
print('target 문장의 정수 인코딩 :',decoder_input[:5])

target 문장의 정수 인코딩 : [[1, 3, 48, 52, 3, 4, 3, 2], [1, 3, 39, 52, 69, 54, 59, 56, 14, 3, 2], [1, 3, 31, 65, 3, 69, 66, 72, 71, 56, 3, 4, 3, 2], [1, 3, 28, 66, 72, 58, 56, 3, 4, 3, 2], [1, 3, 45, 52, 63, 72, 71, 3, 4, 3, 2]]


위 과정까지 훈련 데이터 준비 완료\
아래부터는 학습에서 실제로 필요한 label 데이터 구축\
왜 필요한가?\
<교사 강요>\
예측값을 계속 넣고 학습시킨다면 잘못된 길로 빠질 수 있음\
따라서 정확한 값인 label을 넣고 학습시킨다

In [16]:
# label을 구축하기
# 실제값에는 <sos> 토큰 제거
# 시작할 때 <sos>를 입력으로 받되 출력은 단어이어야 함

decoder_target = []

# 프랑스어 list 돌기
for line in lines.tar:
  timestep = 0
  encoded_line = []

  # 라인에 있는 문자마다 본다
  for char in line:

    # 시점이 0 이상이라면
    if timestep > 0:

      # 정수 인코딩을 수행한 결과를 encoded_list에 넣는다
      encoded_line.append(tar_to_index[char])

    # 시점을 1 증가시킴
    timestep = timestep + 1
  decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩 :',decoder_target[:5])

target 문장 레이블의 정수 인코딩 : [[3, 48, 52, 3, 4, 3, 2], [3, 39, 52, 69, 54, 59, 56, 14, 3, 2], [3, 31, 65, 3, 69, 66, 72, 71, 56, 3, 4, 3, 2], [3, 28, 66, 72, 58, 56, 3, 4, 3, 2], [3, 45, 52, 63, 72, 71, 3, 4, 3, 2]]


In [17]:
# padding을 위한 문장 길이 확인
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)

source 문장의 최대 길이 : 22
target 문장의 최대 길이 : 76


In [18]:
# padding
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [19]:
# one - hot encoding
# 문자 단위 번역기이기 때문에 워드 임베딩은 별도로 사용되지 않음
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

### 모델

In [22]:
# 영어를 입력 받아야하니까 영어 사전 크기로
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True) # 은닉층 노드 수 = 256, 마지막 hs만 반환

# encoder_outputs은 여기서는 불필요
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# LSTM은 RNN과는 달리 상태가 두 개. 은닉 상태와 셀 상태. = context vector
encoder_states = [state_h, state_c]

In [24]:
# 프랑스어로 출력해야하니까 프랑스어 사전 크기로
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True) # 은닉층 노드 수 = 256

# 디코더에게 인코더의 은닉 상태, 셀 상태를 전달, input data를 넘겨줌.(initial_state=encoder_states)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)

# 분류해야할 것은 단어사전 만큼의 크기, 따라서 단어 사전 크기로 class개수 지정
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')

# softmax 통과
decoder_outputs = decoder_softmax_layer(decoder_outputs)

# input = encoder에서 입력하는 값(영어 사전 크기의 벡터)(인코더의 입력)
          #decoder에서 입력하는 값(프랑스어 사전 크기의 벡터)(디코더의 입력)
# output = 디코더의 최종 출력 - softmax를 통과하여 영어 - 프랑스어 변환이 된 결과
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# optimizer: RMSProp , loss function: Cross entropy
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [25]:
# 모델 학습
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=40, validation_split=0.2)

Epoch 1/40
750/750 [==============================] - 22s 17ms/step - loss: 0.8540 - val_loss: 0.7754
Epoch 2/40
750/750 [==============================] - 10s 14ms/step - loss: 0.5740 - val_loss: 0.6601
Epoch 3/40
750/750 [==============================] - 10s 14ms/step - loss: 0.5034 - val_loss: 0.5973
Epoch 4/40
750/750 [==============================] - 10s 14ms/step - loss: 0.4563 - val_loss: 0.5545
Epoch 5/40
750/750 [==============================] - 12s 15ms/step - loss: 0.4230 - val_loss: 0.5196
Epoch 6/40
750/750 [==============================] - 10s 14ms/step - loss: 0.3964 - val_loss: 0.4949
Epoch 7/40
750/750 [==============================] - 10s 14ms/step - loss: 0.3748 - val_loss: 0.4712
Epoch 8/40
750/750 [==============================] - 12s 16ms/step - loss: 0.3576 - val_loss: 0.4522
Epoch 9/40
750/750 [==============================] - 10s 14ms/step - loss: 0.3434 - val_loss: 0.4412
Epoch 10/40
750/750 [==============================] - 11s 14ms/step - loss: 0.331

### 모델을 동작시켜보자! - 입력한 문장에 대해 번역하게 하기

In [26]:
# 인코더 정의
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

# 디코더 정의
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,)) # hs 기억하기
decoder_state_input_c = Input(shape=(256,)) # 셀 상태 기억하기
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c] # decoder의 input 정제

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c] # 리스트로 저장
decoder_outputs = decoder_softmax_layer(decoder_outputs) # softxmax 통과
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [27]:
# 인덱스를 통해 단어를 얻을 수 있는 영어, 프랑스어 사전
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [28]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  # 입력에 대해 encoder가 예측한다.
  states_value = encoder_model.predict(input_seq)

  # 디코더 초기 입력 설정
  # <SOS>에 해당하는 원-핫 벡터 생성 - 최초 시점
  # 프랑스어 사전 크기의 0벡터 생성 - 디코더에 입력할 시퀀스
  target_seq = np.zeros((1, 1, tar_vocab_size))
  # /t가 나타나는 위치에 1을 할당 - 문장에 끝에 1을 할당
  target_seq[0, 0, tar_to_index['\t']] = 1.

  stop_condition = False
  decoded_sentence = ""

  # stop_condition이 True가 될 때까지 루프 반복
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    # 예측 token과 해당 시점의 hs, 셀 상태 또한 반환 받음
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 문자로 변환
    # 문장 속에서 문자의 확률을 예측한 값 중에 가장 값이 높은 인덱스를 선택한다.
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    # 인덱스를 문자로 변환한다
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence += sampled_char

    # <eos>에 도달하거나 최대 길이를 넘으면 중단.
    if (sampled_char == '\n' or
        len(decoded_sentence) > max_tar_len):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    # 0벡터로 초기화
    target_seq = np.zeros((1, 1, tar_vocab_size))
    # 원핫 벡터이기 때문에 이전에 등장한 인덱스에 1을 지정해주면 됨
    target_seq[0, 0, sampled_token_index] = 1.

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

In [29]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
  input_seq = encoder_input[seq_index:seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장:', lines.src[seq_index])
  print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
  print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력

1/1 [==============================] - 0s 30ms/step
-----------------------------------
입력 문장: Go.
정답 문장: Bouge ! 
번역 문장: Au retournée ! 
1/1 [==============================] - 0s 19ms/step
-----------------------------------
입력 문장: Hello!
정답 문장: Bonjour ! 
번역 문장: Salut ! 
1/1 [==============================] - 0s 20ms/step
-----------------------------------
입력 문장: Got it!
정답 문장: J'ai pigé ! 
번역 문장: Attendez ! 
1/1 [==============================] - 0s 22ms/step
-----------------------------------
입력 문장: Go home.
정답 문장: Rentre à la maison. 
번역 문장: Vas ! 
1/1 [==============================] - 0s 19ms/step
-----------------------------------
입력 문장: Get going.
정답 문장: En avant. 
번역 문장: Déplacez-vous. 


## Word-Level 번역기

In [ ]:
# data import는 이전과 동일한 과정을 거침
# 따라서 수행하지는 않으나 혹시 몰라서 적어놓음
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def download_zip(url, output_path):
    response = requests.get(url, headers=headers, stream=True)
    if response.status_code == 200:
        with open(output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"ZIP file downloaded to {output_path}")
    else:
        print(f"Failed to download. HTTP Response Code: {response.status_code}")

url = "http://www.manythings.org/anki/fra-eng.zip"
output_path = "fra-eng.zip"
download_zip(url, output_path)

path = os.getcwd()
zipfilename = os.path.join(path, output_path)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [32]:
# text preprocessing
def to_ascii(s):
  # 프랑스어 악센트(accent) 삭제
  # 예시 : 'déjà diné' -> deja dine
  return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sent):
  # 악센트 제거 함수 호출
  sent = to_ascii(sent.lower()) # 소문자 변환

  # 단어와 구두점 사이에 공백 추가.
  # ex) "I am a student." => "I am a student ."
  sent = re.sub(r"([?.!,¿])", r" \1", sent)

  # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환.
  sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

  # 다수 개의 공백을 하나의 공백으로 치환
  sent = re.sub(r"\s+", " ", sent)
  return sent

In [37]:
# 전처리 테스트
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"

print('전처리 전 영어 문장 :', en_sent)
print('전처리 후 영어 문장 :',preprocess_sentence(en_sent))
print('전처리 전 프랑스어 문장 :', fr_sent)
print('전처리 후 프랑스어 문장 :', preprocess_sentence(fr_sent))

전처리 전 영어 문장 : Have you had dinner?
전처리 후 영어 문장 : have you had dinner ?
전처리 전 프랑스어 문장 : Avez-vous déjà diné?
전처리 후 프랑스어 문장 : avez vous deja dine ?


In [38]:
# 33000개 샘플 사용
num_samples = 33000

# 사용할 33000개의 sample에 대한 전처리 수행
def load_preprocessed_data():

  # encoder의 입력, decoder의 입력, decoder의 lable(교사 강요)
  encoder_input, decoder_input, decoder_target = [], [], []

  with open("fra.txt", "r") as lines:
    for i, line in enumerate(lines):
      # source 데이터와 target 데이터 분리
      src_line, tar_line, _ = line.strip().split('\t') # tab을 기준으로 line split

      # source 데이터 전처리
      src_line = [w for w in preprocess_sentence(src_line).split()] # 전처리 후 공백 기준으로 word split

      # target 데이터 전처리
      tar_line = preprocess_sentence(tar_line) # 이전에 정의한 전처리 함수 호출

      # <sos> 토큰 붙여주기 - 문장의 시작
      # 굳이 입력할 때는 eos를 넣어줄 필요가 없음, 예측할 것이 eos이고 이것이 문장의 끝이기 때문에
      tar_line_in = [w for w in ("<sos> " + tar_line).split()]
      # <eos> 토큰 붙여주기 - 문장의 끝
      tar_line_out = [w for w in (tar_line + " <eos>").split()]

      # 전처리한 데이터 저장하기
      encoder_input.append(src_line)
      decoder_input.append(tar_line_in)
      decoder_target.append(tar_line_out)

      # 문장 끝까지 다 돌면
      if i == num_samples - 1:
        break

  return encoder_input, decoder_input, decoder_target

In [39]:
# 상위 5개만 sample 출력해보기
# dataset 지정 안해줘도 내부에서 txt 파일 호출하여 원하는 수(33000)만큼 전처리
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()
print('인코더의 입력 :',sents_en_in[:5])
print('디코더의 입력 :',sents_fra_in[:5])
print('디코더의 레이블 :',sents_fra_out[:5])

인코더의 입력 : [['go', '.'], ['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.']]
디코더의 입력 : [['<sos>', 'va', '!'], ['<sos>', 'marche', '.'], ['<sos>', 'en', 'route', '!'], ['<sos>', 'bouge', '!'], ['<sos>', 'salut', '!']]
디코더의 레이블 : [['va', '!', '<eos>'], ['marche', '.', '<eos>'], ['en', 'route', '!', '<eos>'], ['bouge', '!', '<eos>'], ['salut', '!', '<eos>']]


Q. 현재 시점의 디코더 셀의 입력은 오직 이전 디코더 셀의 출력을 입력으로 받는다고 했는데 디코더의 입력에 해당하는 데이터인 sents_fra_in이 왜 필요할까?\
A. 훈련 과정에서는 이전 시점의 디코더 셀의 출력을 현재 시점의 디코더 셀의 입력으로 넣어주지 않고, 이전 시점의 실제값을 현재 시점의 디코더 셀의 입력값으로 하는 방법을 사용한다. (교사 강요)

In [40]:
# 영어 단어 집합 생성
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
encoder_input = pad_sequences(encoder_input, padding="post")

# 프랑스어 단어 집합 생성
tokenizer_fra = Tokenizer(filters="", lower=False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)

# 각 문장을 정수 인덱싱 및 패딩 처리
decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_input = pad_sequences(decoder_input, padding="post")

decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)
decoder_target = pad_sequences(decoder_target, padding="post")

In [41]:
print('인코더의 입력의 크기(shape) :',encoder_input.shape)
print('디코더의 입력의 크기(shape) :',decoder_input.shape)
print('디코더의 레이블의 크기(shape) :',decoder_target.shape)

인코더의 입력의 크기(shape) : (33000, 7)
디코더의 입력의 크기(shape) : (33000, 16)
디코더의 레이블의 크기(shape) : (33000, 16)


In [42]:
# 예제랑 단어집합 크기가 다름
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1
print("영어 단어 집합의 크기 : {:d}, 프랑스어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

영어 단어 집합의 크기 : 4481, 프랑스어 단어 집합의 크기 : 7873


In [43]:
# 단어 -> 정수 리스트 / 정수 -> 단어 리스트  각각 생성
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word
tar_to_index = tokenizer_fra.word_index
index_to_tar = tokenizer_fra.index_word

In [44]:
# 데이터 섞기

# encoder_input과 동일한 크기의 인텍스 배열을 생성
indices = np.arange(encoder_input.shape[0])

# 인덱싱 된 배열을 무작위로 섞기
np.random.shuffle(indices)
print('랜덤 시퀀스 :',indices)

랜덤 시퀀스 : [20600 15582   232 ...  2734  7091 27667]


In [47]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

print(encoder_input[30997])
print(decoder_input[30997]) # <sos> 존재
print(decoder_target[30997]) # <eos> 존재

# decoder_input과 decoder_target은 앞에 붙은 <sos> 토큰과 뒤에 붙은 <eos>을 제외하면 동일한 정수 시퀀스를 가져야 함

[   5  335 3142    1    0    0    0]
[   2    7    8  324    8 4643    1    0    0    0    0    0    0    0
    0    0]
[   7    8  324    8 4643    1    3    0    0    0    0    0    0    0
    0    0]


In [48]:
# 검증 데이터: 10프로
n_of_val = int(33000*0.1)
print('검증 데이터의 개수 :',n_of_val)

검증 데이터의 개수 : 3300


In [49]:
# split
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [50]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)

훈련 source 데이터의 크기 : (29700, 7)
훈련 target 데이터의 크기 : (29700, 16)
훈련 target 레이블의 크기 : (29700, 16)
테스트 source 데이터의 크기 : (3300, 7)
테스트 target 데이터의 크기 : (3300, 16)
테스트 target 레이블의 크기 : (3300, 16)


In [52]:
embedding_dim = 64  # 임베딩 벡터 차원
hidden_units = 64  # LSTM 은닉 상태 크기

In [53]:
# 인코더
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(hidden_units, return_state=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # encoder 출력 값(필요하진 않음), 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

In [54]:
# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, hidden_units) # 임베딩 층, 프랑스어 - 은닉층 크기
dec_emb = dec_emb_layer(decoder_inputs)
dec_masking = Masking(mask_value=0.0)(dec_emb) # 패딩 0은 연산에서 제외

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용 -> context vector(연결되는 다리)
decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
# 디코더는 다중 클래스 분류 문제를 해결하는 중
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# 모델의 입력과 출력을 정의.
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 손실함수: Cross entropy
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [55]:
# 입력으로 영어 사전, <sos>토큰이 있는 프랑스어 사전, <eos>토큰이 있는 프랑스어 사전을 각각 넘겨준다.(label)
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50)

Epoch 1/50
233/233 [==============================] - 34s 106ms/step - loss: 3.3680 - acc: 0.6144 - val_loss: 2.0467 - val_acc: 0.6188
Epoch 2/50
233/233 [==============================] - 9s 38ms/step - loss: 1.8559 - acc: 0.6642 - val_loss: 1.7622 - val_acc: 0.7381
Epoch 3/50
233/233 [==============================] - 8s 36ms/step - loss: 1.6488 - acc: 0.7449 - val_loss: 1.6064 - val_acc: 0.7473
Epoch 4/50
233/233 [==============================] - 8s 33ms/step - loss: 1.5112 - acc: 0.7575 - val_loss: 1.4957 - val_acc: 0.7623
Epoch 5/50
233/233 [==============================] - 7s 31ms/step - loss: 1.4078 - acc: 0.7697 - val_loss: 1.4056 - val_acc: 0.7749
Epoch 6/50
233/233 [==============================] - 9s 37ms/step - loss: 1.3128 - acc: 0.7892 - val_loss: 1.3287 - val_acc: 0.7891
Epoch 7/50
233/233 [==============================] - 8s 35ms/step - loss: 1.2375 - acc: 0.8002 - val_loss: 1.2697 - val_acc: 0.7977
Epoch 8/50
233/233 [==============================] - 7s 31ms/step 

### 동작 시키기
<전체적인 번역 과정>
1. 번역하고자 하는 입력 문장이 인코더로 입력되어 인코더의 마지막 시점의 은닉 상태와 셀 상태를 얻는다.
2. 인코더의 은닉 상태와 셀 상태, 그리고 토큰 <sos>를 디코더로 보낸다.
3. 디코더가 토큰 <eos>가 나올 때까지 다음 단어를 예측하는 행동을 반복한다.

In [56]:
# 인코더
encoder_model = Model(encoder_inputs, encoder_states)

# 디코더 설계 시작
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(hidden_units,)) # hs 저장
decoder_state_input_c = Input(shape=(hidden_units,)) # 셀 상태 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c] # 두 가지 합쳐서 리스트로

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

# 수정된 디코더
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [57]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 정수 생성
  target_seq = np.zeros((1,1))
  target_seq[0, 0] = tar_to_index['<sos>']

  stop_condition = False
  decoded_sentence = ''

  # stop_condition이 True가 될 때까지 루프 반복
  # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용 - decoder에게는 당연
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 단어로 변환
    # 리스트에서 가장 확률이 높은 인덱스를 가져온다.
    sampled_token_index = np.argmax(output_tokens[0, -1, :])

    # 가장 확률이 높은 인덱스를 문자로 변환한다.
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 단어를 예측 문장에 추가
    decoded_sentence += ' '+sampled_char

    # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
    if (sampled_char == '<eos>' or
        len(decoded_sentence) > 50):
        stop_condition = True

    # 상태 초기화: 0벡터
    target_seq = np.zeros((1,1))

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq[0, 0] = sampled_token_index

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

In [58]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0):
      sentence = sentence + index_to_src[encoded_word] + ' '
  return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0 and encoded_word != tar_to_index['<sos>'] and encoded_word != tar_to_index['<eos>']):
      sentence = sentence + index_to_tar[encoded_word] + ' '
  return sentence

In [59]:
for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)

1/1 [==============================] - 0s 20ms/step
입력문장 : do come again . 
정답문장 : revenez nous voir . 
번역문장 : viens encore y aller . 
--------------------------------------------------
1/1 [==============================] - 0s 22ms/step
입력문장 : you re very tall . 
정답문장 : vous etes tres grande . 
번역문장 : vous etes tres grande . 
--------------------------------------------------
1/1 [==============================] - 0s 21ms/step
입력문장 : what s that ? 
정답문장 : qu est ce la ? 
번역문장 : qu est ce que c est ? 
--------------------------------------------------
1/1 [==============================] - 0s 23ms/step
입력문장 : i was impressed . 
정답문장 : j ai ete impressionnee . 
번역문장 : j ai ete impressionne . 
--------------------------------------------------
1/1 [==============================] - 0s 24ms/step
입력문장 : who developed it ? 
정답문장 : qui l a developpee ? 
번역문장 : qui l a batie ? 
--------------------------------------------------


In [60]:
for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_test[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_test[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_test[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)

1/1 [==============================] - 0s 21ms/step
입력문장 : let s watch tv . 
정답문장 : allons regarder la television . 
번역문장 : prenons le l air . 
--------------------------------------------------
1/1 [==============================] - 0s 21ms/step
입력문장 : i m a hard worker . 
정답문장 : je suis un bourreau de travail . 
번역문장 : je suis vraiment une chanson . 
--------------------------------------------------
1/1 [==============================] - 0s 24ms/step
입력문장 : it s all over now . 
정답문장 : maintenant tout est fini . 
번역문장 : c est tout que nous en aller a fait . 
--------------------------------------------------
1/1 [==============================] - 0s 19ms/step
입력문장 : tom waved . 
정답문장 : tom a fait signe de la main . 
번역문장 : tom a vu tom . 
--------------------------------------------------
1/1 [==============================] - 0s 26ms/step
입력문장 : wait and see . 
정답문장 : attendons en buvant du the . 
번역문장 : allez vous en train de travailler ! 
------------------------------------------